In [ ]:
from __future__ import division, print_function

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

# Try to import safe_rl from system
# if it fails get it from the main folder directly instead.
import utilities
safe_learning = utilities.import_from_directory('safe_learning', '../')

### Test the triangulation

Create a triangulation on a grid and plot the corresponding results.
Then take a test point and return in which simplex the corresponding point lies

The naming and indexing convention follows the standard C-indexing style (last index moves the fastest).

The space is split up into regular hyperrectangles, as given by the limits and npoints arguments. These rectangles are indexed from 0 onwards. Within each hyperrectangle is a delaunay triangulation, which is computed only for one of them (the code is basically a wrapper around ```scipy.spatial.Delaunay```.).

Within each hyperrectangle, the order of the triangle indeces is determined by the Delaunay decomposition. Outside, it is given by ```#rectangle * nsimplex + #simplex```, where ```#rectangle``` is the index of the rectangle, ```nsimplex``` is the number of simpleces per hyperrectangle, and ```#simplex``` is the index of the simplex within the rectangle, as given by the Delaunay decomposition.

In [ ]:
limits = [[-1, 1], [-1, 1]]
npoints = [3, 4]

# Create triangulation
tri = safe_learning.Triangulation(limits, npoints)

In [ ]:
# Get all simplices and points
simplices = tri.simplices(np.arange(tri.nsimplex))
points = tri.index_to_state(np.arange(tri.nindex))

# colors
simplex_color = 'g'
node_color = 'b'
rect_color = 'k'

# Plot nodes
plt.plot(points[:, 0], points[:, 1], 'o', color=node_color)

# Annotate nodes
for p in points:
    plt.text(p[0] - 0.03, p[1] + 0.03,
             tri.state_to_index(p)[0],
             ha='right', color=node_color)

# Annotate rectangles
rectangles = tri.rectangle_to_state(np.arange(tri.nrectangles))
rectangles += tri.unit_maxes / 2
for i, point in enumerate(rectangles):
    plt.text(point[0], point[1], '#%d' % i,
             ha='center', color=rect_color)

# Plot triangles
plt.triplot(points[:, 0], points[:, 1], simplices.copy(),
            color=simplex_color)

# Annotate triangles
for i, simplex in enumerate(simplices):
    p = points[simplex].mean(axis=0)
    plt.text(p[0], p[1], '#%d' % i,
             ha='center', color=simplex_color)

# Format plot nicely
offset = 0.1 * np.array([-1, 1])
plt.xlim(tri.limits[0] + offset * np.diff(tri.limits[0]))
plt.ylim(tri.limits[1] + offset * np.diff(tri.limits[1]))


# test points
test_point = np.array([[0.9, 0.9],
                       [-0.5, -0.2]])
plt.plot(test_point[:, 0], test_point[:, 1], 'x')
plt.show()

# Output the result of the evaluation
test_simplices = tri.find_simplex(test_point)
test_simplex_nodes = tri.simplices(test_simplices)

print('Evaluation results for test points:\n')
for index, nodes in zip(test_simplices, test_simplex_nodes):
    print("simplex {id} with nodes {nodes}".format(id=index, nodes=nodes))


### Naive implementation in scipy

This is just the vanilla scipy implementation of Delaunay, not really something that is supported by this library in any way.

In [ ]:
from safe_learning.functions import ScipyDelaunay

tri = ScipyDelaunay(limits, npoints)
points = tri.points
simplices = tri.simplices

# Plot nodes
plt.plot(points[:, 0], points[:, 1], 'o', color=node_color)

# Annotate nodes
for i, p in enumerate(points):
    plt.text(p[0] - 0.03, p[1] + 0.03, i,
             ha='right', color=node_color)
    
# Plot triangles
plt.triplot(points[:, 0], points[:, 1], simplices.copy(),
            color=simplex_color)

# Annotate triangles
for i, simplex in enumerate(simplices):
    p = points[simplex].mean(axis=0)
    plt.text(p[0], p[1], '#%d' % i,
             ha='center', color=simplex_color)
    
# Format plot nicely
offset = 0.1 * np.array([-1, 1])
plt.xlim(tri.limits[0] + offset * np.diff(tri.limits[0]))
plt.ylim(tri.limits[1] + offset * np.diff(tri.limits[1]))
plt.show()

### Speed comparison

In [ ]:
import timeit

test_points = np.arange(10, 51, 5)
ntrials = 5
time = np.empty((len(test_points), 2), dtype=np.float)

for i, n in enumerate(test_points):
    npoints = [n, n]

    time[i, 0] = timeit.timeit('Triangulation({0}, {1})'.format(limits, npoints),
                               setup='from safe_learning import Triangulation',
                               number=ntrials)

    time[i, 1] = timeit.timeit('ScipyDelaunay({0}, {1})'
                               .format(limits, npoints),
                               setup='from safe_learning.functions import ScipyDelaunay',
                               number=ntrials)

ntriangles = 2 * test_points ** 2

plt.plot(ntriangles, time[:, 0], label='efficient')
plt.plot(ntriangles, time[:, 1], label='scipy')
plt.ylabel('Computation time [s]')
plt.xlabel('Number of triangles')
plt.legend()
plt.show()

# Test the function value capability

#### Get matrix for values of test points

For each vertex x, the next state is given by some function $f(x)$. The goal is to find the value $V(f(x))$ as a function of the values at the other vertices. This value can then be used as a constraint in the optimization problem.
$$V(f(x)) = a^\mathrm{T} V(\mathrm{vertices})$$

We start by finding the simplex corresponding to $f(x)$. Given this simplex's vertices, we solve for $a$ above, using one arbitrary vertex as a reference point:
$$V(f(x)) = V(v_1) + a_1 V(v_2) + a_2 V(v_3)$$

The hyperplane spanned by the simplex is given by
\begin{align*}
\xi_1 ( x_2 - x_1) + \xi_2 (y_2 - y_1) = V(v_2) - V(v_1) \\
\xi_1 ( x_3 - x_1) + \xi_2 (y_3 - y_1) = V(v_3) - V(v_1) 
\end{align*}

and, as a result,
$$V(f(x_t)) = V(v_1) + (x_t - x_1, y_t - y_1)
\left( \begin{matrix}
x_2 - x_1 & y_2 - y_1 \\
x_3 - x_1 & y_3 - y_1
\end{matrix} \right)^{-1}
\left( \begin{matrix}
V(v_2) - V(v_1) \\
V(v_3) - V(v_1)
\end{matrix} \right),
$$
which can easily be solved for $a_1$ and $a_2$.

In [ ]:
fig = plt.figure()
ax = Axes3D(fig)
ax.view_init()
c = ax.plot_trisurf(points[:, 0], points[:, 1], 2 * np.random.rand(len(points)),
                    triangles=simplices.copy(),
                    cmap='viridis', lw=0.1)
plt.colorbar(c)